INSTALL PYTUBE

In [1]:

## Install pyTube3 https://github.com/nficano/pytube
!pip install git+https://github.com/pytube/pytube


  Cloning https://github.com/pytube/pytube to c:\users\ayool\appdata\local\temp\pip-req-build-2aumvzje
  Running command git clone -q https://github.com/pytube/pytube 'C:\Users\ayool\AppData\Local\Temp\pip-req-build-2aumvzje'


In [2]:

#importing the module 
from pathlib import Path
from pytube import YouTube
video2Index = YouTube('https://www.youtube.com/watch?v=ijtKxXiS4hE').streams.first().download()
video_name = Path(video2Index).stem



CREATE AN AZURE VIDEO INDEXER INSTANCE

In [3]:
pip install video-indexer

Note: you may need to restart the kernel to use updated packages.


INITIALIZE CLIENT

In [6]:
from video_indexer import VideoIndexer

vi = VideoIndexer(vi_subscription_key='5e7d57663db14ee58a06385263836438',
                  vi_location='trial',
                  vi_account_id='b8f7e815-9386-40e3-a527-2a6743630cef')

UPLOAD VIDEO

In [7]:
video_id = vi.upload_to_video_indexer(
              input_filename = video2Index,
              video_name=video_name, #must be unique
              video_language='English')

Getting video indexer access token...


SSLError: HTTPSConnectionPool(host='api.videoindexer.ai', port=443): Max retries exceeded with url: /auth/trial/Accounts/b8f7e815-9386-40e3-a527-2a6743630cef/AccessToken?allowEdit=true (Caused by SSLError(SSLError(1, '[SSL: WRONG_VERSION_NUMBER] wrong version number (_ssl.c:1091)')))

GET VIDEO INFO

In [10]:
info = vi.get_video_info(video_id, video_language='English')


Getting video info for: 5cf2436441
Video still processing, current status: 25%


EXTRACT KEYFRAMES ID

In [46]:
keyframes = []
for shot in info["videos"][0]["insights"]["shots"]:
    for keyframe in shot["keyFrames"]:
        keyframes.append(keyframe["instances"][0]['thumbnailId'])

GET KEYFRAMES ID

In [48]:
for keyframe in keyframes:
    img_str = vi.get_thumbnail_from_video_indexer(video_id,    
                                                  keyframe)
img_str

Getting thumbnail from video: 5c1c2b8d69, thumbnail: 2feb5c5e-bf61-4542-b5f9-af0c587b335f
Getting thumbnail from video: 5c1c2b8d69, thumbnail: 2e226c69-9b27-4076-b404-8ad45353f8db
Getting thumbnail from video: 5c1c2b8d69, thumbnail: f3d0131a-f9fd-48fe-8aed-87bb11968da5
Getting thumbnail from video: 5c1c2b8d69, thumbnail: ce457d1e-04a7-4ee8-99fa-4ca53ad1ca7b
Getting thumbnail from video: 5c1c2b8d69, thumbnail: d9c97c2a-3486-4ab8-8fa3-6d1ef07e1b8d
Getting thumbnail from video: 5c1c2b8d69, thumbnail: 23a10025-b2ef-463c-8e39-571f2fcd422e
Getting thumbnail from video: 5c1c2b8d69, thumbnail: 0c9c5cba-6534-4009-bb4f-1aa80c9d3828
Getting thumbnail from video: 5c1c2b8d69, thumbnail: 70285405-16ee-4291-ae6a-10bc030f613a
Getting thumbnail from video: 5c1c2b8d69, thumbnail: 1508a28f-6ecb-49be-af13-eb1fcec3f751
Getting thumbnail from video: 5c1c2b8d69, thumbnail: df33ebb5-d4a8-49ef-9427-b37449178f70
Getting thumbnail from video: 5c1c2b8d69, thumbnail: 271d3b37-c99a-498e-9b53-8d21c7ef5702
Getting th

\x84ET\x85\x1bk\xc2\xa0\x92\x070\xc5\xf5e\xd0\xe5\xb0RM=I]EU]A\xbdMm\\\xaa\xa8\xab\xa8<\xf2J\xbe\xf1\xde\x16\x1eFM\xc3\xd2\x95z\xb4_\xec\xeaN\x1f\xbb&\xbf)\x15\xb6\xdbm\xc2\xb5j4\xea\xe7\xdb\x84]\xaf\xe5tGKW\xdb\x18\xe0\x92\x15\xc1\x94\xd7\x85\xa1q\x85\xc7,\xd4\xd2\xf7\x92S\x8b\n\xe3R/\xb7h\xbb\xf2\xd9\\\x91\x8a\x1aq\x88\x03\x80\x009IN\xc3n{\x17\xee$\x911\xedY\x00?9?g\xfb\x1f[Q:\xe1\x13\xe9S"\x8a\xd3\rD\x91\xa1*V\xd5a\x16\x00}\xdd\x8e\xab\xef7[\xac<z\x9cL=\x9d2\xb9\x96\xcb\x96lyw\x13\xee\x94#C\x88\xd3\x9a\x8d\xec\xed\x97\xaa\xe4.n$\x96,\x11\xc9s\'\xe5\xa1\x08\xda\xa9V\xbd\x98G\xbd\xcei\xfbC]OON\xb5E\x96S\xc5\x14q*8\x0e\x92\xa1a\x00\x03y\x14\x9c)\'\xf6F\xc6\xa4\xe5\x1d\x9e\xc9OZ\xc2\x88p\xedJ\xe6\x94\xa8\x83\xfb!Gj\xbdQw\xaauf\xbd\xa3\x94C\x97E-5"\xb6K]*0\xa9H:\xfa\xba\x15c\x7f\xdaV\xa6l\xe5\xbb\xa9>\x1e\xddK\xbfh\xbby\x0f}Ge8\xc6\xa6\xe9E\xaa7\x9cT\xba;y\x11\x99\xa4\xb4\xf1b\xca\xa8\n\xbb\xca\n\xcc*O~E\xf2\xe0R\xce\xd2\xb5z\\\x809\x0c\x8b3\xac\x9eU\x95.\x96:x\x00\x89\x01\x00 \xe3\x1a\x

Use the Azure Computer Vision API to Extract Popular Brands from Key Frames

INSTALL COMPUTERVISION CLIENT

In [6]:
pip install --upgrade azure-cognitiveservices-vision-computervision

Note: you may need to restart the kernel to use updated packages.


In [50]:
pip install pillow

Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the 'c:\Users\ayool\Desktop\VideoIndexingproject\clayvideo\Scripts\python.exe -m pip install --upgrade pip' command.


AUTHRNTICATE THE CLIENT

In [11]:
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
from azure.cognitiveservices.vision.computervision.models import VisualFeatureTypes
from msrest.authentication import CognitiveServicesCredentials

from array import array
import os
from PIL import Image
import sys
import time



GET VARIABLES FOR ENDPOINT AND APIKEYS

In [12]:
subscription_key_CV = "c31039d57019480baf596f1ae6acdfa8"
endpoint_CV = "https://cognitivegeneral.cognitiveservices.azure.com/"
computervision_client = ComputerVisionClient(endpoint_CV, CognitiveServicesCredentials(subscription_key_CV))

VISUALIZE KEYFRAMES

In [13]:
import time
from IPython.display import display
from PIL import Image
import io

timeout_interval, timeout_time = 5, 10.0
image_features = ["brands"]
for index, keyframe in enumerate(keyframes):
    if index % timeout_interval == 0:
        print("Trying to prevent exceeding request limit waiting {}  seconds".format(timeout_time))
        time.sleep(timeout_time)
# Get KeyFrame Image Byte String From Video Indexer
img_str = vi.get_thumbnail_from_video_indexer(video_id, keyframe)
# Convert Byte Stream to Image Stream
img_stream = io.BytesIO(img_str)
# Analyze with Azure Computer Vision
cv_results = computervision_client.analyze_image_in_stream(img_stream, image_features)
print("Detecting brands in keyframe {}: ".format(keyframe))
if len(cv_results.brands) == 0:
    print("No brands detected.")
else:
    for brand in cv_results.brands:
        print("'{}' brand detected with confidence {:.1f}% at location {}, {}, {}, {}".format( brand.name, brand.confidence * 100, brand.rectangle.x, brand.rectangle.x + brand.rectangle.w, brand.rectangle.y, brand.rectangle.y + brand.rectangle.h))

NameError: name 'keyframes' is not defined

 Key Frames to Azure Custom Vision API to Custom Detect Brands¶


In [59]:
!pip install azure-cognitiveservices-vision-customvision


You should consider upgrading via the 'c:\users\ayool\desktop\videoindexingproject\clayvideo\scripts\python.exe -m pip install --upgrade pip' command.


INITIALIZE CUSTOM VISION CLIENT

In [79]:
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from msrest.authentication import ApiKeyCredentials
prediction_threshold = .8
prediction_key =  "ee27e51576d049fdb0d180bb91f1fa89"
ENDPOINT = "https://cvtestayo-prediction.cognitiveservices.azure.com/"
project_id = "a6161fd5-c077-45f6-987f-0b486aa07557"
published_name = "Iteration1"
credentials = ApiKeyCredentials(in_headers={"Prediction-key": prediction_key})
predictor = CustomVisionPredictionClient(endpoint=ENDPOINT, credentials=credentials)

PREDICT KEYFRAMES WITH CUSTOM VISION MODEL

In [80]:

import time
timeout_interval, timeout_time = 5, 10.0
image_features = ["brands"]

for index, keyframe in enumerate(keyframes):
    if index % timeout_interval == 0:
        print("Trying to prevent exceeding request limit waiting {} seconds".format(timeout_time))
        time.sleep(timeout_time)
    # Get KeyFrame Image Byte String From Video Indexer
    img_str = vi.get_thumbnail_from_video_indexer(video_id, keyframe)
    # Convert Byte Stream to Image Stream
    img_stream = io.BytesIO(img_str)  
    # Analyze with Azure Computer Vision
    cv_results = predictor.detect_image(project_id, published_name, img_stream)
    predictions = [pred for pred in cv_results.predictions if pred.probability > prediction_threshold]
    print("Detecting brands in keyframe {}: ".format(keyframe))
    if len(predictions) == 0:
        print("No custom brands detected.")
    else:
        for brand in predictions:
            print("'{}' brand detected with confidence {:.1f}% at location {}, {}, {}, {}".format( \
                brand.tag_name, brand.probability * 100, brand.bounding_box.left, brand.bounding_box.top, \
                brand.bounding_box.width, brand.bounding_box.height))

Trying to prevent exceeding request limit waiting 10.0 seconds
Getting thumbnail from video: 5c1c2b8d69, thumbnail: 2feb5c5e-bf61-4542-b5f9-af0c587b335f


CustomVisionErrorException: Bad Request Image Format, Uri: 5b90a1572a5644768a5b28b6fd4fe87c